In [1]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
# read data
processed_cleveland <- read_csv("original data/processed.cleveland.data", col_names = F)

ERROR: Error: 'data/processed.cleveland.data' does not exist in current working directory ('/home/jovyan/work/Group_Project/heart disease data').


In [3]:
#rename variables
colnames(processed_cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

processed_cleveland

ERROR: Error: object 'processed_cleveland' not found


In [ ]:
#create the TRAIN SET and TEST SET
set.seed(2000)

cleveland_filter <- processed_cleveland |>
            select(age, chol, thalach, num) |>
            mutate(num = as_factor(num)) |>
            sample_n(300)
cleveland_filter

cleveland_split <- initial_split(cleveland_filter, prop = 0.75, strata = num)
cleveland_train<- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

# cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = num)




In [ ]:
cleveland_recipe <- recipe(num ~ ., data = cleveland_train) |>
                    step_scale(all_predictors()) |>
                    step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

# gridvals = tibble(neighbors = seq(from = 1, to = 80, by = 5))

set.seed(2000)

cleve_workflow <- workflow() |>
            add_recipe(cleveland_recipe) |>
            add_model(knn_spec) |>
            fit(data = cleveland_train)

            # tune_grid(resamples = cleveland_vfold, grid = gridvals) 


In [ ]:
# options(repr.plot.height = 5, repr.plot.width = 6)
# cross_val_plot <- cleve_workflow |>
#                 ggplot(aes(x = neighbors, y = mean)) +
#                 geom_point() +
#                 labs(x = "Neighbors", y = "Accuracy")
# cross_val_plot